<a href="https://colab.research.google.com/github/jmelm93/semrush-api-backfill-site-data/blob/main/Backfill_SEMRush_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import logging
import urllib
import pandas as pd
from datetime import datetime, date
import dateutil.relativedelta

today = date.today().strftime("%Y-%m-%d") 

In [ ]:
apikey = "XXXXXXXX"
columns = "Xn,Dn,Rk,Or,Ot,Oc,FKn,FPn"
domain_list=["hp.com","intuit.com"]
database="us"
end_date = today
start_date_months_back = 6

In [ ]:
shifted_timestamp = pd.Timestamp(today) - pd.DateOffset(months=end_date_months_back)
start_date = shifted_timestamp.strftime("%Y-%m-%d") 

date_list=pd.date_range(start_date,end_date, 
              freq='MS').strftime("%Y%m15").tolist()

date_list

['20210515', '20210615', '20210715', '20210815', '20210915', '20211015']

In [ ]:
# https://www.semrush.com/api-analytics/#rank

def get_report_data(apikey,domain,database,columns,date):
    report = requests.get(
        url="https://api.semrush.com",
        params={
            "type": "domain_rank",
            "key": apikey,
            "domain": domain,
            "database": database,
            "export_escape": 1,
            "export_decode": 1,
            "display_date": date.replace('-',''),
            "export_columns": columns
        }
    )
    logging.info(report.status_code)
    return report.content


In [ ]:
def parse_response(data):
    results = []
    data = data.decode('unicode_escape')
    lines = data.split('\r\n')
    lines = list(filter(bool, lines))
    columns = lines[0].split(';')

    for line in lines[1:]:
        result = {}
        for i, datum in enumerate(line.split(';')):
            result[columns[i]] = datum.strip('"\n\r\t')
        results.append(result)

    return results

In [ ]:
# parsed_data = parse_response(data=data)
# pd.DataFrame(parsed_data)

In [ ]:
results = []
for domain in domain_list:
  for date in date_list:
    data = get_report_data(apikey=apikey,database=database,domain=domain,columns=columns,date=date)
    parsed_data = parse_response(data=data)
    df = pd.DataFrame(parsed_data)
    df['date']=date
    results.append(df)


In [ ]:
results

[   Domain Rank Organic Keywords Organic Traffic Organic Cost      date
 0  hp.com  316          1655068        12898684     14047096  20210515,
    Domain Rank Organic Keywords Organic Traffic Organic Cost      date
 0  hp.com  306          1764076        13103424     14212803  20210615,
    Domain Rank Organic Keywords Organic Traffic Organic Cost      date
 0  hp.com  321          1885162        12959591     13513657  20210715,
    Domain Rank Organic Keywords Organic Traffic Organic Cost      date
 0  hp.com  318          1865414        12908670     13489326  20210815,
    Domain Rank Organic Keywords Organic Traffic Organic Cost      date
 0  hp.com  342          1872201        12474597     14111277  20210915,
 Empty DataFrame
 Columns: [date]
 Index: [],
        Domain Rank Organic Keywords Organic Traffic Organic Cost      date
 0  intuit.com  164          1300299        20658930     56189312  20210515,
        Domain Rank Organic Keywords Organic Traffic Organic Cost      date


In [ ]:
pd.concat(results)

,Domain,Rank,Organic Keywords,Organic Traffic,Organic Cost,date
0,hp.com,316,1655068,12898684,14047096,20210515
0,hp.com,306,1764076,13103424,14212803,20210615
0,hp.com,321,1885162,12959591,13513657,20210715
0,hp.com,318,1865414,12908670,13489326,20210815
0,hp.com,342,1872201,12474597,14111277,20210915
0,intuit.com,164,1300299,20658930,56189312,20210515
0,intuit.com,170,1273898,20460838,57411389,20210615
0,intuit.com,231,1197372,16636637,65359732,20210715
0,intuit.com,212,1281761,17254424,66309328,20210815
0,intuit.com,225,1254378,17008932,32218115,20210915
